<a href="https://colab.research.google.com/github/abxda/python/blob/main/UP_Semana_5_Viernes_y_S%C3%A1bado_01_Noticias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U duckduckgo_search
!pip install wordcloud matplotlib

In [ ]:
# Instalar spaCy
!pip install -U spacy

# Descargar e instalar el modelo en español
!python -m spacy download es_core_news_sm


In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from duckduckgo_search import AsyncDDGS
import asyncio
import pandas as pd
import string
from nltk.corpus import stopwords
import nltk
import spacy

In [ ]:

# Configuraciones para la búsqueda de noticias en Aguascalientes
configuraciones = [
    {"idioma": "Español", "keywords": "Aguascalientes", "region": "mx-es"}
]

# Diccionario para almacenar los resultados de noticias por idioma
resultados_noticias = {"Español": []}

async def buscar_noticias(idioma, keywords, region):
    async_ddgs = AsyncDDGS()
    noticias = await async_ddgs.news(keywords=keywords, region=region, safesearch='moderate', max_results=100)
    return idioma, noticias

async def main():
    tasks = [buscar_noticias(conf['idioma'], conf['keywords'], conf['region']) for conf in configuraciones]
    results = await asyncio.gather(*tasks)
    for idioma, noticias in results:
        resultados_noticias[idioma].extend(noticias)  # Añadir noticias a la lista correspondiente del idioma
        print(f"Resultados para {idioma}: {len(noticias)} noticias encontradas.")

    # Imprimir las listas de noticias por idioma
    for idioma, lista_noticias in resultados_noticias.items():
        print(f"\n{idioma} ({len(lista_noticias)} noticias):")
        print([noticia['title'] for noticia in lista_noticias[:10]])  # Mostrar solo los títulos de las primeras 10 noticias para brevedad

    # Opcional: Concatenar títulos de noticias y generar nube de palabras
    news_text = ' '.join([news['body'] for news in resultados_noticias['Español']])
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(news_text)

    # Mostrar la nube de palabras
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()

# Luego en otra celda de Jupyter:
await main()


In [ ]:
len(resultados_noticias["Español"])

In [ ]:
df_noticias = pd.DataFrame(resultados_noticias["Español"])

In [ ]:
df_noticias

In [ ]:
df_noticias.columns

In [ ]:
# Primero, asegúrate de descargar las stopwords
nltk.download('stopwords')

In [ ]:
nltk.download('punkt')

In [ ]:
df_noticias['clean_body'] = df_noticias['body'].str.lower()
df_noticias['clean_body'] = df_noticias['clean_body'].str.translate(str.maketrans('', '', string.punctuation))
stop_words = set(stopwords.words('spanish'))
df_noticias['clean_body'] = df_noticias['clean_body'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

In [ ]:
def generar_nube_de_palabras(textos_limpios):

    # Unir todos los cuerpos de texto en una sola cadena
    textos_unidos = ' '.join(textos_limpios)

    # Crear la nube de palabras
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(textos_unidos)

    # Mostrar la nube de palabras
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()

In [ ]:
generar_nube_de_palabras(df_noticias['clean_body'])

In [ ]:

# Contar noticias por fuente
conteo_fuentes = df_noticias['source'].value_counts()

# Crear gráfica de barras
plt.figure(figsize=(12, 8))  # Ajusta el tamaño para mejor visualización
conteo_fuentes.plot(kind='bar', color='skyblue')  # Puedes cambiar el color si deseas
plt.title('Cantidad de Noticias por Fuente')
plt.xlabel('Fuente')
plt.ylabel('Cantidad de Noticias')
plt.xticks(rotation=90)  # Rota las etiquetas del eje x a 90 grados
plt.tight_layout()  # Ajusta automáticamente los parámetros de la subtrama para dar espacio al contenido
plt.show()

In [ ]:
nlp = spacy.load("es_core_news_sm")

# Tokenización y lematización
df_noticias['lemmatized'] = df_noticias['clean_body'].apply(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))

In [ ]:
generar_nube_de_palabras(df_noticias['lemmatized'])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans

In [ ]:
# Preprocesar el texto
def preprocess(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

df_noticias['processed'] = df_noticias['body'].apply(preprocess)


In [ ]:
# Vectorización con TF-IDF usando n-gramas de caracteres de 3 a 5 longitudes
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(3, 5))
X = vectorizer.fit_transform(df_noticias['processed'])

# Normalización de vectores
X_normalized = normalize(X)

In [ ]:
# Clustering
db = DBSCAN(eps=0.5, min_samples=2, metric='cosine')
clusters = db.fit_predict(X_normalized)

In [ ]:
# Clustering con K-means
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(X_normalized)

In [ ]:
df_noticias['cluster'] = clusters

# Análisis de clusters
cluster_counts = df_noticias['cluster'].value_counts()

# Imprimir los resultados de los clusters más grandes
print(cluster_counts)


In [ ]:
clusters

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

def generar_nube_de_palabras(textos_limpios, ax):
    # Unir todos los cuerpos de texto en una sola cadena
    textos_unidos = ' '.join(textos_limpios)

    # Crear la nube de palabras
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(textos_unidos)

    # Mostrar la nube de palabras en el eje especificado
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.axis('off')

# Crear una figura con subplots
fig, axs = plt.subplots(5, 2, figsize=(20, 50))  # 5 filas, 2 columnas

# Generar una nube de palabras y listar títulos para cada cluster
for i in range(5):
    # Filtrar el DataFrame para obtener textos y títulos del cluster actual
    cluster_data = df_noticias[df_noticias['cluster'] == i]
    textos_cluster = cluster_data['processed'].tolist()
    titulos_cluster = cluster_data[['title', 'source']].apply(lambda x: f"{x['title']} ({x['source']})", axis=1).tolist()

    if textos_cluster:  # Verificar si la lista no está vacía
        generar_nube_de_palabras(textos_cluster, axs[i, 0])
        # Poner títulos en la segunda columna
        axs[i, 1].text(0.5, 0.5, "\n".join(titulos_cluster), verticalalignment='center', horizontalalignment='left', fontsize=12, family='monospace')
        axs[i, 1].axis('off')

    axs[i, 0].set_title(f'Cluster {i+1} - Nube de Palabras')
    axs[i, 1].set_title(f'Cluster {i+1} - Títulos y Fuentes')

plt.tight_layout(pad=3.0)
plt.show()

